# Atividade 1
## Item 3 - Problema MAX-QBF com Set Cover

In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import sys
import io

In [2]:
# geração de instâncias aleatórias

def gerar_subconjuntos(n):
    N = set(range(1, n+1))
    S = []
    
    cobertos = set()
    for i in range(n):
        tamanho = random.randint(2, n)  # tamanho aleatório >=2
        subconjunto = set(random.sample(range(1, n+1), tamanho))
        S.append(subconjunto)
        cobertos.update(subconjunto)
    
    # garante cobertura
    faltando = N - cobertos
    if faltando:
        S[-1] = S[-1].union(faltando)
    
    return S


def gerar_instance(n):
    S = gerar_subconjuntos(n)
    
    linhas = []
    linhas.append(str(n))  # número de elementos
    linhas.append(" ".join(str(len(s)) for s in S))  # tamanhos dos subconjuntos
    
    for s in S:
        linhas.append(" ".join(map(str, sorted(s))))  # elementos de cada subconjunto
    
    # segunda parte -> sequências aleatórias com tamanhos decrescentes
    number = n
    while number >= 1:
        linha = " ".join(str(random.randint(-10, 10)) for _ in range(number))
        linhas.append(linha)
        number -= 1
    
    return "\n".join(linhas)

In [3]:
def solve_max_sc_qbf_linearized(input_stream, output_file_name=None):
    """
    Resolve o problema MAX-QBF com Set Cover (MAX-SC-QBF)
    utilizando a linearização da função objetivo quadrática,
    direcionando a saída para um arquivo, se especificado.

    Args:
        input_stream: Um objeto de fluxo de entrada (ex: sys.stdin ou um arquivo aberto).
        output_file_name: O nome do arquivo para onde a saída deve ser direcionada.
                          Se None, a saída será para o console (sys.stdout).
    """
    # Armazenar o stdout original para restaurá-lo depois
    original_stdout = sys.stdout
    output_file = None

    try:
        # Se um nome de arquivo de saída for fornecido, abrir o arquivo e redirecionar sys.stdout
        if output_file_name:
            output_file = open(output_file_name, 'w')
            sys.stdout = output_file # Redireciona os prints do Python para o arquivo

        # 1. Leitura dos dados da instância conforme Item 3.1 da Atividade [1, 2]
        n = int(input_stream.readline().strip()) # Número de variáveis binárias / subconjuntos / elementos [1]
        # Ler a linha de contagens de elementos por subconjunto (não usado diretamente no modelo, mas parte do formato) [1]
        _ = input_stream.readline().strip().split()
        # Ler as listas de elementos cobertos por cada subconjunto [1]
        # subsets_covered_elements[i] conterá uma lista de elementos (0-indexados) cobertos pelo subconjunto i.
        subsets_covered_elements = []
        for _ in range(n):
            line_elements = list(map(int, input_stream.readline().strip().split()))
            # Ajustar elementos para 0-indexados, já que a entrada é 1-indexada [informação fora das fontes, mas comum em programação]
            subsets_covered_elements.append([e - 1 for e in line_elements])

        # Reconstruir a matriz A (assumindo formato triangular superior) [1]
        A = [[0.0] * n for _ in range(n)]
        # A entrada para a matriz A é triangular superior [1].
        # Ela é lida linha por linha, preenchendo a diagonal e acima.
        for i in range(n):
            # Ler os coeficientes da linha atual (do a_ii até a_in)
            line_coeffs = list(map(float, input_stream.readline().strip().split()))
            col_idx_in_line = 0
            for j in range(i, n): # j começa de i para preencher a diagonal e acima
                A[i][j] = line_coeffs[col_idx_in_line]
                col_idx_in_line += 1

        # 2. Criar um novo modelo Gurobi
        model = gp.Model("MAX_SC_QBF_Linearized")
        # 3. Adicionar as variáveis binárias originais (x_i)
        # x[i] representa a i-ésima variável binária, indicando a seleção do subconjunto (i+1) [3]
        x = model.addVars(n, vtype=gp.GRB.BINARY, name="x")
        # 4. Adicionar as variáveis de linearização (y_ij)
        # y[i, j] representa o produto x_i * x_j.
        # Como A é triangular superior, só precisamos de y_ij para i <= j.
        y = model.addVars(n, n, vtype=gp.GRB.BINARY, name="y")
        # 5. Definir a função objetivo linearizada

        # A função objetivo original do MAX-QBF é Z = ∑_{i=1 to n} ∑_{j=1 to n} a_ij * x_i * x_j [4]
        # Com a linearização y_ij = x_i * x_j, e considerando que A[i][j] é 0 para i > j (matriz triangular superior),
        # a função objetivo se torna linear: Z = ∑_{i=0 to n-1} ∑_{j=i to n-1} A[i][j] * y[i, j]
        obj = gp.quicksum(A[i][j] * y[i, j] for i in range(n) for j in range(n) if i <= j)
        model.setObjective(obj, gp.GRB.MAXIMIZE)
        # 6. Adicionar as restrições de linearização
        # Para cada par (i, j) onde y_ij é definido (i <= j), adicionamos as três restrições para garantir y_ij = x_i * x_j.

        for i in range(n):
            for j in range(i, n): # Iterar sobre i <= j
                # Restrição 1: y_ij <= x_i
                model.addConstr(y[i, j] <= x[i], f"linear_y{i}_{j}_le_x{i}")
                # Restrição 2: y_ij <= x_j
                model.addConstr(y[i, j] <= x[j], f"linear_y{i}_{j}_le_x{j}")
                # Restrição 3: y_ij >= x[i] + x[j] - 1
                model.addConstr(y[i, j] >= x[i] + x[j] - 1, f"linear_y{i}_{j}_ge_sum_minus_1")
        # Para i == j, estas restrições forçam y[i,i] = x[i], o que é correto para x_i * x_i = x_i.
        # 7. Adicionar as restrições de cobertura de conjuntos (Set Cover) [3]

        # Para todo elemento k_element no universo N (de 0 a n-1 em nossa indexação interna),
        # deve existir ao menos um subconjunto S_i selecionado (x_i=1) tal que k_element ∈ S_i.
        for k_element in range(n): # k_element representa o índice do elemento (0 a n-1)
            # Encontrar quais subconjuntos cobrem este elemento k_element
            covering_subsets_for_k = []
            for i_subset in range(n): # i_subset representa o índice do subconjunto (0 a n-1)
                if k_element in subsets_covered_elements[i_subset]:
                    covering_subsets_for_k.append(x[i_subset])

        # Adicionar a restrição de que a soma dos x_i dos subconjuntos que cobrem k_element deve ser >= 1
        model.addConstr(gp.quicksum(covering_subsets_for_k) >= 1, f"cover_element_{k_element+1}")
        # Configurar limite de tempo (10 minutos conforme a Atividade 1, seção 4.2) [5]
        model.Params.TimeLimit = 600 # segundos

        # MUITO IMPORTANTE: Direcionar a saída de log do Gurobi para o arquivo especificado
        if output_file_name:
            model.Params.LogFile = output_file_name

        # 8. Otimizar o modelo
        print("Iniciando otimização do MAX-SC-QBF (Modelo Linearizado)...")
        model.optimize()
        # 9. Obter e exibir os resultados
        print("\n" + "=" * 40)
        print("Resultados da Otimização (MAX-SC-QBF Linearizado):")

        print("=" * 40)
        if model.status == gp.GRB.OPTIMAL:
            print(f"Status: Ótimo (Optimal)")
            print(f"Valor da Solução Ótima (Z): {model.ObjVal:.4f}")
            print("Valores das Variáveis x (Subconjuntos Selecionados):")
            selected_subsets = []
            for i in range(n):
                print(f" x[{i+1}] = {int(x[i].X)}") # Apresentando como x1 a xn
                if x[i].X > 0.5: # Usar uma tolerância para valores binários
                    selected_subsets.append(i + 1)
            print(f"Subconjuntos selecionados: {selected_subsets}")
        elif model.status == gp.GRB.TIME_LIMIT:
            print(f"Status: Limite de tempo atingido (Time Limit)")

            print(f"Melhor Solução Encontrada (Z): {model.ObjVal:.4f}")
            print(f"Gap de Otimalidade: {model.MIPGap * 100:.2f}%")
            print("Valores das Variáveis x na melhor solução encontrada:")
            selected_subsets = []
            for i in range(n):
                print(f" x[{i+1}] = {int(x[i].X)}")
                if x[i].X > 0.5:
                    selected_subsets.append(i + 1)
            print(f"Subconjuntos selecionados: {selected_subsets}")
        elif model.status == gp.GRB.INFEASIBLE:
            print("Status: Inviável (Infeasible)")
        else:
            print(f"Status da Otimização: {model.status}")

    finally:
        # Sempre restaurar o stdout original e fechar o arquivo, se ele foi aberto
        if output_file:
            sys.stdout = original_stdout
            output_file.close()

In [4]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
with open('instancia' + str(n) + 'variaveis.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(example_instance_content)

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content), "log" + str(n) + "variaveis.txt")

Digite o valor de n:  5


--- Executando com n fornecido acima ---


In [5]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
with open('instancia' + str(n) + 'variaveis.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(example_instance_content)

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content), "log" + str(n) + "variaveis.txt")

Digite o valor de n:  25


--- Executando com n fornecido acima ---


In [6]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
with open('instancia' + str(n) + 'variaveis.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(example_instance_content)

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content), "log" + str(n) + "variaveis.txt")

Digite o valor de n:  50


--- Executando com n fornecido acima ---


In [7]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
with open('instancia' + str(n) + 'variaveis.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(example_instance_content)

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content), "log" + str(n) + "variaveis.txt")

Digite o valor de n:  100


--- Executando com n fornecido acima ---


In [8]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
with open('instancia' + str(n) + 'variaveis.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(example_instance_content)

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content), "log" + str(n) + "variaveis.txt")

Digite o valor de n:  200


--- Executando com n fornecido acima ---


In [9]:
n = int(input("Digite o valor de n: "))
example_instance_content = gerar_instance(n)
with open('instancia' + str(n) + 'variaveis.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(example_instance_content)

print("--- Executando com n fornecido acima ---")
solve_max_sc_qbf_linearized(io.StringIO(example_instance_content), "log" + str(n) + "variaveis.txt")

Digite o valor de n:  400


--- Executando com n fornecido acima ---
